# TensorFlow hands-on!

Hello and welcome to the hands-on portion of the **Image Recognition** workshop. By now, you should have a good understanding of how Convolutional Neural Networks work and what TensorFlow is. We'll now move to building an image recognition model. 

### The Case 
Today we will be working on a simple case - classifying images from the Nederlandse Vereniging Makelaars. The classes include binnen, buiten, and mappen. Like this: 

<img src='labels.jpg'>


### Outline
<img src='outline2.jpg' align=left>
<br><br><br><br><br>
<br><br><br><br><br>

### Processing Images

The first thing that we need to do is to import and process our images. 

#### What does processing mean? 
When we need to build our computational graph, we have to define what the input data shape is. THis is a core component of TensorFlow - in that before you do anything, you define an abstract architecture for processing vectors (tensors). 

In our case, we are going to reshape our images to 256x256 pixels! 

Like this: 
<img src='compressed.jpg'>


### Importing images 
Luckily, we can import the images from their .jpg format and convert them into a processed array with just a few lines of code. Let's move to the code to get an idea as to what we are doing. 

## Python Packages 

Before we do anything in our script, let's import the packages we will be using throughout this hands-on excercise. To get an idea what these packages do, see this: 

<img src='packages2.jpg' align=left width=400 >

In [2]:
##################################################
########### Don't change this code ###############
##################################################

# For accessing files and storing information
import numpy as np 
import glob

# To mediate impatience
import progressbar 

# For visualization
import matplotlib.pyplot as plt
from matplotlib import patches

# For encoding labels 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# For converting images to numeric values
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Some functions we'll need 
from tools import one_hot_encoder, batch_norm_layer, random_batch

# For building our model 
import tensorflow as tf 

##################################################
##################################################
##################################################

Using TensorFlow backend.


## Importing Images

Let's get our images into our working directory! 

To do so, define some list objects that will contain the path to each image in your images directory. You can do this by calling:
<br>
```
# Fill list with paths of images
image_paths = glob.glob('path/to/images/*.jpg')

```

**Remember to do this for each class! **


Next, we need to combine all of our lists with our class filepaths. You can do this by calling: 
<br>
```
# Combine lists of class file paths 
all_paths = np.concatenate([class1_paths, class2_paths, class3_paths])
```

<br>
Then let's print the file paths! 

In [3]:
##################################################
################### Start Here ###################
##################################################

# Fill list with paths to photos 
binnen_paths = glob.glob('images/binnen/*.jpg')
buiten_paths = glob.glob('images/buiten/*.jpg')
mappen_paths = glob.glob('images/map/*.jpg')

# Combine the data
paths = np.concatenate([binnen_paths, buiten_paths, mappen_paths])

# Print the first 5 values in the array paths
print(paths[:5])


##################################################
##################################################

['images/binnen\\0000002.jpg' 'images/binnen\\0000003.jpg'
 'images/binnen\\0000004.jpg' 'images/binnen\\0000005.jpg'
 'images/binnen\\0000006.jpg']


### Defining the label set

Well now that we have the paths to each class's images, it is quite easy to define the label set. We will do this by creating arrays with the class name, and filling it with that value for the number of iamges we have for that label. 

You can do this by calling: 
<br>
```
class_label = np.array(['class_name'] * len(class_paths))
```

<br> Let's try it out! 

In [4]:
##################################################
################### Start Here ###################
##################################################

# Defining the label set 
binnen_label = np.array(['binnen'] * len(binnen_paths))
buiten_label = np.array(['buiten'] * len(buiten_paths))
mappen_label = np.array(['map'] * len(mappen_paths))

# Combine the data
labels = np.concatenate([binnen_label, buiten_label, mappen_label])

# Print the first 5 labels (should all be binnen)
print(labels[:5])


##################################################
##################################################

['binnen' 'binnen' 'binnen' 'binnen' 'binnen']


### Import images as arrays 

This part is a little bit tricky. 

<br>What we are going to do is call the keras ```load_img``` and ```img_to_array``` functions. We want to store these as *unint8* data type in order to reduce memory usage. We'll append each array to a list, and then turn that list into an array - creating an array of arrays. This should become easier when we print out the shape of the array. 

<br>
Our goal is to make images into something like this: 
<img src='img_to_array.jpg'>


#### For loop
<br> 
If you're familiar with programming fundamentals, you already know what a for loop is. In this example, we need to "loop" over each *path* in our list of filepaths, and turn that image into an array. An example of a simple for loop is as follows:
<br>
``` 
# Loop over all paths in list of file paths
for path in paths:

    # Load each specified image (from the file path)
    image = load_img(path, target=(256, 256))
    
    # Conver the image into an array
    image = img_to_array(image)
    
    # Append to a list
    image_list.append(image)
    
```

<br>

Let's try this out! 

In [5]:
##################################################
########### Don't change this code ###############
##################################################

# Initialize an empty list to fill with image arrays
image_list = []

# Initiate a progress bar
bar = progressbar.ProgressBar(maxval=len(paths)).start()


##################################################
################### Start Here ###################
##################################################


# Loop through the directory and fill list with resized pixel values
for index, path in enumerate(paths): 
    
    # Resizes the image to 256x256x3 resolution
    image = load_img(path, target_size=(256, 256, 3))
    
    # Converts the image object to an array (of type unint8 for efficient storage) 
    image = img_to_array(image).astype(np.uint8)
    
    # Append the array to the image list
    image_list.append(image)
    
    # Update the progressbar
    bar.update(index)
    
# Convert the image list to a numpy array 
image_array = np.array(image_list)

print(image_array.shape)

##################################################
##################################################

(400, 256, 256, 3)


### One-hot encoding
<br>
Right now, our labels object is an array with the labels *binnen*, *buiten*, and *map*. Unforunately, computers are not as smart as we are, and need these words to be represented as numbers in order to process them. 

One-hot encoding is the process of turning labels into a computer-readable format, and looks like this. 
<br>

<img src='hot_encode.jpg'>

<br>

To hot encode, just call our ```one_hot_encoder``` function we imported from our tools library! 

In [6]:
##################################################
################### Start Here ###################
##################################################

# Call one_hot_encoder() function on the labels!
hot_encoded, label_classes = one_hot_encoder(labels)


##################################################
##################################################

### Inputs, outputs, and train-test-split

So now that we have our images and labels stored as numeric vectors - we can assign them to our input $X$ variable, and the labels as our $y$ variable. 

<br>
$X = images$
<br>
$y = labels$
<br>


### Train test split 

#### Why do we perform a train/test split? 
The reason we perform a train/test split is to understand how well our model approximates on a general distribution of inputs. In other words, how do we know if our model will work, if we do not test it on data it has never seen before? 
<br>

<img src='traintest.jpg'>

<br>

To perform a train test split on our dataset, call the ```train_test_split``` function, like this: 
<br>
```
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25) 
```

In [7]:
##################################################
################### Start Here ###################
##################################################

# Define images as input X and labels as output y 
X = image_array 
y = hot_encoded

# Perform a train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)


##################################################
##################################################

# Defining Useful Functions

### Placeholders 

By now, you understand that tensorflow is an abstract computing language. For this reason - we create *placeholders* to feed data into later. This will contribute to our computational graph that we build later on. 


In [8]:
def create_placeholders(n_h0, n_w0, n_c0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_h0 -- scalar, height of an input image
    n_w0 -- scalar, width of an input image
    n_c0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    X_placeholder = tf.placeholder(tf.float32, [None, n_h0, n_w0, n_c0])
    y_placeholder= tf.placeholder(tf.float32, [None, n_y])
    
    return X_placeholder, y_placeholder

### Create the placeholders 

To create the placeholders, you will need scalar inputs representing the shape of your inputs and outputs. To help with this, the annotation can defined as: 
<br>
<br>
<br>
$placeholder = tensor(height, width, channels, numclasses)$
<br>
<br>

You can access the heights, widths, channels, and number of classes by using the following: 
<br>
```
height = X.shape[1]
width = X.shape[2]
channels = X.shape[3]
num_classes = y.shape[1]
```
<br>
Let's try this out! 

In [9]:
##################################################
################### Start Here ###################
##################################################

# Define the height of X
n_h0 = X.shape[1]

# Define the width of X
n_w0 = X.shape[2]

# Define the number of RGB channels 
n_c0 = X.shape[3]

# Define the number of classes
n_y  = y.shape[1]

# Define the X and y placeholders
X_ph, y_ph = create_placeholders(n_h0, n_w0, n_c0, n_y)

# Print the place holders 
print ("X = " + str(X_ph))
print ("Y = " + str(y_ph))


##################################################
##################################################

X = Tensor("Placeholder:0", shape=(?, 256, 256, 3), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)


### Convolutional Layer 

Arguably the most important part of today is in understand what a convolutional layer is. Here is an example that we looked at in the presentation. 

<img src='conv_layer.jpg' align=right>
<br><br>


Fortunately for us, tensorflow makes it very easy to use convolutional layers. See this example for your code. 

```
tf.nn.conv2d(input_data, weights, stride, padding='SAME')
```
<br>
<br>
If you have any questions during this part, now is the time to ask! 

In [10]:
##################################################
################### Start Here ###################
##################################################

def convolutional_layer(input_data, 
                        num_channels, 
                        num_filters,
                        filter_shape,
                        name):
    """ 
    Function used to prepare convolutional layers for image recognition training. 
    
    Arguments: 
    input_date   : input vector (beginning with X input vector and then subsequent layer)
    num_channels : the number of vectors corresponding to RGB values (=3)
    num_filters  : the number of filters to apply to convolve the pixel arrays
    filter_shape : the shape of the filters used to convolve the pixel arrays
    pool_shape   : the shape and method for pooling
    
    Returns:
    An Output Layer
    
    """
    
    # FILL IN CODE HERE
    # Define the filter shape for the convolutions (filter_shape x num_channels x num_filters)
    conv_filter_shape = [filter_shape[0], filter_shape[1], num_channels, num_filters]
    
    
    # FILL IN CODE HERE
    # Initialize weights to assign to the filters 
    weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.03), name=name+'_W')
    
    # Defualt value
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')
    
    # Default value
    stride = [1, 1, 1, 1]
    
    # FILL IN CODE HERE - if you have trouble, type ?tf.nn.conv2d for hints
    # Setup the convolutional layer operation
    output_layer = tf.nn.conv2d(input_data, weights, stride, padding='SAME')

    
    # Add the bias
    output_layer += bias

    
    # Apply a ReLU non-linear activation
    output_layer = tf.nn.relu(output_layer)
    
    return output_layer


##################################################
##################################################

### Max Pool Layer 

Fortunately, max pool is much easier to understand. 

<img src='maxpool.jpg' align=left>

In [11]:
##################################################
################### Start Here ###################
##################################################

def max_pool_layer(input_data, pool_shape):
    
    # Perform Max Pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    
    strides = [1, 2, 2, 1]
    
    output_layer = tf.nn.max_pool(input_data, ksize=ksize, strides=strides, padding='SAME')
    
    return output_layer


##################################################
##################################################

# Building Computational Graph 


### What computation graph do we want? 
<br> 

We want to build our convolutional neural network architecture. We are going to use a very simple, 3-layer convolutional network which hopefully will work on our dataset. The model should look like this: 

<img src='architecture.jpg'>


### Computational Graph
When we define our convolutional and maxpool layers in the next step, what we are essentially doing is telling TensorFlow that this is the model we want to feed our data through. What is represented in the image is therefore interpreted as tensors in the computational graph.

In [12]:
# Define the first convolution
# Use 6 filters! 
conv_1 = convolutional_layer(X_ph, 3, 6, [3, 3], name='conv_1')

# Define the first maxpool 
maxpool_1 = max_pool_layer(conv_1, [2, 2])

# Define the first convolution
# Use 12 filters! 
conv_2 = convolutional_layer(maxpool_1, 6, 12, [3, 3], name='conv2')

# Define the second maxpool 
maxpool_2 = max_pool_layer(conv_2, [2, 2])

# Define the third convolution
# Use 6 filters 
conv_3 = convolutional_layer(maxpool_2, 12, 6, [3, 3], name='conv3')

# Define the third maxpool 
maxpool_3 = max_pool_layer(conv_3, [2, 2])

# Flatten the last maxpool layer 
flattened = tf.reshape(maxpool_3, [-1, 6144])

### Fully-Connected Layers 

For the fully connected layers, don't worry too much about the code.

In [13]:
# setup some weights and bias values for this layer, then activate with ReLU
fc_weights_1 = tf.Variable(tf.truncated_normal([6144, 200], stddev=0.03), name='fc_weights_1')
fc_bias_1 = tf.Variable(tf.truncated_normal([200], stddev=0.01), name='fc_bias_1')
fc_layer_1 = tf.matmul(flattened, fc_weights_1) + fc_bias_1
fc_layer_1 = tf.nn.relu(fc_layer_1)

In [14]:
# another layer with softmax activations
fc_weights_2 = tf.Variable(tf.truncated_normal([200, 3], stddev=0.03), name='fc_weights_2')
fc_bias_2 = tf.Variable(tf.truncated_normal([3], stddev=0.01), name='fc_bias_2')
fc_layer_2 = tf.matmul(fc_layer_1, fc_weights_2) + fc_bias_2
y_ = tf.nn.softmax(fc_layer_2)

### Define the Loss Measurement 
#### Note on Learning

The way that neural networks learn is via something known as *gradient descent*. What this means is that after a forward pass through the network, the model goes **back**, and measures whether or not the weights and biases helped or hurt the overall loss score. It does so by using the *chain rule* (from high school calculus), and taking steps closer to the global minimum of the loss function. 
<br>

<img src='gradient.jpg'>

<br>
For our case, we are going to define the $J()$ function (the loss function) as cross-entropy loss. Don't worry too much about this!  

#### Note on Optimization 
2014 was a great year for deep learning. This guy from the University of Amsterdam discoverd the *de facto* optimisation algorithm, known as Adam. 

<br>
<img src='adam.jpg'>

In [15]:
# Define the loss function - J()
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc_layer_2, labels=y_ph))


# Initiate an Adam Optimizer
optimiser = tf.train.AdamOptimizer(learning_rate=.0003).minimize(cross_entropy)


# Define an accuracy score function (to print while we are training)
correct_prediction = tf.equal(tf.argmax(y_ph, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# Training the model 
### Batch processing 
Because deep learning can be veryyyyyyy slow - sometime we need to split up our data in order to process it faster. This is known as minibatch processing, and works by essentially sending small amounts of data through the network, allow it to perform gradient descent (adjusting the weights), and then send the next batch. 

I like to think of this as a rapid-fire approach. Send through lots of little chunks of data so that we now how are model is doing. Once all of our batches are complete, we say that the model has completed an *epoch*

### Initialize the Tensors!

Tensorflow requires that we specify when we are intiializing the tensors defined above. We'll do this for you, but just a heads up. 


In [16]:
# Define a some batches from our training set 
batches = random_batch(X_train, y_train, batch_size=64)

# Initialize tensors! 
init_op = tf.global_variables_initializer()

## Last, but not least

Let's train this thing! I'm going to walk you through this part, so don't worry too much about the lack of hints. 

In [17]:
# Initiate empty list to fill with costs 
costs = []

# Initiate empty list to fill with accuracies 
accuracies = []

# Initiate a progressbar 
bar = progressbar.ProgressBar(maxval=len(batches)).start()

# Set a random seed
np.random.seed(42)

# Begin a tensorflow session
with tf.Session() as sess:
    
    # (Actually) initialise the variables
    sess.run(init_op)
    
    
    # START CODE HERE
    # Define a for loop to iterate through 30 epochs
    for epoch in range(30):
        
        # Iterate through each batch
        for index, batch in enumerate(batches):
            
            # Define the input and outputs from the batch
            X_batch, y_batch = batch
            
            # Initiate the average cost at 0
            avg_cost = 0
        
            # Run an Adam optimizer and minimize the cross-entropy lost 
            # Don't forget to actually feed our data!
            _, cost = sess.run([optimiser, cross_entropy], 
                                feed_dict={X_ph: X_batch, 
                                           y_ph: y_batch})
            
            
            # Add the cost from the first batch
            avg_cost += cost
            costs.append(avg_cost)
            
            # Update the progressbar 
            bar.update(index)
        
        # Get he accuracy on the test set 
        test_accuracy = sess.run(accuracy, 
                                 feed_dict=
                                 {X_ph: X_test, 
                                  y_ph: y_test})        
        
        
        
        # Append the accuracies list with the test accuracy 
        accuracies.append(test_accuracy)
    
        # Print out the information
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), "test accuracy: {:.3f}".format(test_accuracy))
    
    print("Overall Test Accuracy " + "%.2f" % (sess.run(accuracy, feed_dict={X_ph: X_test, y_ph: y_test})*100) + '%')
    
    # In case we want to look at our predictions later 
    predictions = sess.run(y_, feed_dict={X_ph:X_test})

Epoch: 1 cost = 0.894 test accuracy: 0.560
Epoch: 2 cost = 0.809 test accuracy: 0.820
Epoch: 3 cost = 0.756 test accuracy: 0.840
Epoch: 4 cost = 0.614 test accuracy: 0.840
Epoch: 5 cost = 0.480 test accuracy: 0.840
Epoch: 6 cost = 0.402 test accuracy: 0.840
Epoch: 7 cost = 0.367 test accuracy: 0.870
Epoch: 8 cost = 0.342 test accuracy: 0.900
Epoch: 9 cost = 0.342 test accuracy: 0.920
Epoch: 10 cost = 0.420 test accuracy: 0.950
Epoch: 11 cost = 0.420 test accuracy: 0.950
Epoch: 12 cost = 0.189 test accuracy: 0.950
Epoch: 13 cost = 0.182 test accuracy: 0.910
Epoch: 14 cost = 0.176 test accuracy: 0.950
Epoch: 15 cost = 0.136 test accuracy: 0.940
Epoch: 16 cost = 0.117 test accuracy: 0.960
Epoch: 17 cost = 0.086 test accuracy: 0.960
Epoch: 18 cost = 0.066 test accuracy: 0.950
Epoch: 19 cost = 0.047 test accuracy: 0.960
Epoch: 20 cost = 0.035 test accuracy: 0.950
Epoch: 21 cost = 0.028 test accuracy: 0.950
Epoch: 22 cost = 0.023 test accuracy: 0.950
Epoch: 23 cost = 0.020 test accuracy: 0.9